In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlphw1/train.csv
/kaggle/input/nlphw1/test.csv
/kaggle/input/nlphw1/sample_submission.csv


https://stackoverflow.com/questions/40178364/using-regex-to-remove-digits-from-string
https://stackoverflow.com/questions/51194627/python-naive-bayes-with-cross-validation-using-gaussiannb-classifier

In [2]:
a1 = pd.read_csv('../input/nlphw1/train.csv', encoding = "latin-1")
b1 = pd.read_csv('../input/nlphw1/test.csv')

In [3]:
a= a1.drop(['Id'], axis=1)
b = b1.drop(['id'], axis=1)
print(a)

                                                   text  Target
0     @USAirways  ! THE WORST in customer service. @...      -1
1     @united call wait times are over 20 minutes an...      -1
2     @JetBlue what's up with the random delay on fl...      -1
3     @AmericanAir Good morning!  Wondering why my p...       0
4     @united UA 746. Pacific Rim and Date Night cut...      -1
...                                                 ...     ...
7315                            @AmericanAir followback       0
7316  @united thanks for the help. Wish the phone re...       1
7317  @usairways the. Worst. Ever. #dca #customerser...      -1
7318  @nrhodes85: look! Another apology. DO NOT FLY ...      -1
7319  @united you are by far the worst airline. 4 pl...      -1

[7320 rows x 2 columns]


In [4]:
## checking length of train and test data
print('Train length:', len(a))
print('Test length:', len(b))


Train length: 7320
Test length: 7320


# tokenization, stemming

In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
l=[]
#for train data
for i in range(0,7320):
    train = re.sub('[@$?.!#()+\b\d+\b]', ' ', a['text'][i] )
    train=train.lower() #lower case
    train=train.split() # word tokenizing
    ps=PorterStemmer()
    train=[ps.stem(word) for word in train if not word in set(stopwords.words('english'))]
    train=' '.join(train)
    l.append(train)
print(l)


['usairway worst custom servic usairway call month book flight poorcustomerservic usairwaysffly', 'unit call wait time minut airport wait time longer', "jetblu what' random delay flight chanc fals alarm", 'americanair good morn wonder pre-tsa check board pass morn', 'unit ua pacif rim date night cut constantli randomly, one spot, repeat', 'southwestair - still know ski cannot get call back bag svc airport left sever msg', 'unit see told repeatedli pilot late flight kept get late flightr boarded, defibril issu', 'southwestair flt san jose, ca -&gt; denver, co delay hr easili best flight touchdown year fli keep', "yo internjohnradio mrerickv give southwestair plane back they'r mad &amp; threaten take away companion pass luv", "americanair somehow dfw mia got bump b way g what'", 'unit took care rent car drive hour retriev belong due unit error', 'americanair stuck gate miami jfk flight go get tonight frustrat', 'unit loung heathrow use airnzusa http://t co/ hjucp l', 'americanair respons

In [6]:

#for test data
d = []
for i in range(0,7320):
    test = re.sub('[@$?.!#()+\b\d+\b]', ' ', b['text'][i] )
    test=test.lower()  
    test=test.split()
    ps=PorterStemmer()
    test=[ps.stem(word) for word in test if not word in set(stopwords.words('english'))]
    test=' '.join(test)
    d.append(test)
print(d)


["americanair car gng dfw pull hr ago - ici road on-hold aa sinc hr can't reach arpt aa wat", 'americanair all, plane didn\x89ûªt land ident wors condit grk accord metar', "southwestair can't believ mani pay custom left high dri reason flight cancel flightlat monday bdl wow", 'usairway legitim say would rather driven cross countri flown us airway', 'americanair still respons aa great job guy', 'unit develop fli tmrw morn w/ min layover, earlier flight hr layover, move', 'usairway hello anyon', 'usairway husainhaqqani mr husain u shld protest well one ur parti member rehman malik delay pia flight hour', 'usairway likely, flightawar say plane still durango depart', 'americanair even give option hold say line busi plz tri late flightr', 'unit announc pre board address mobil disabl requir travel lot stuff preboard', 'usairway realli embarrass ask complimentari drink/snack detail here: https://t co/ za xb h &amp; argu', 'southwestair passport time trip could still fli photo id thingsishould

# Count Vectorizer

In [7]:
#count vectorizer for train data
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
X=cv.fit_transform(l)
y=a.iloc[:,1].values

In [8]:
#count vectorization for test data
from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(max_features=5000)
X_t=cv.fit_transform(d)

# Splitting Training data

In [9]:
# splitting trainning data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

# Logistic Regression

In [10]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter = 100000).fit(X_train, y_train)

cv_scores = cross_val_score(clf, X_train, y_train,scoring='f1_macro')

# Mean Cross validation Score
print("Mean Cross-validation scores: {}".format(cv_scores.mean()))
print()

# Print Co-efficients
print("Logistic.coef_:", clf.coef_)
print("Logistic.intercept_:", clf.intercept_)

# Check test data set performance
print("Logistic Performance: ", clf.score(X_test,y_test))


Mean Cross-validation scores: 0.7012308534846875

Logistic.coef_: [[-0.41359655 -0.08615254  0.01002051 ... -0.25478917 -0.06084385
  -0.26045726]
 [ 0.55326902  0.1217586  -0.39289146 ...  0.39405044  0.37724249
  -0.08025673]
 [-0.13967246 -0.03560607  0.38287095 ... -0.13926127 -0.31639863
   0.340714  ]]
Logistic.intercept_: [-0.01206694  0.79541577 -0.78334883]
Logistic Performance:  0.7725409836065574


# XGBOOST

In [11]:
## XGboost
from  xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
xgbc= XGBClassifier(random_state=42)
xgbc_param = {
               'max_depth' : [4,5,6],
              'learning_rate' : [0.1,0.5,0.8],
                'n_estimators' : [100,150]
             }
grid_xgb = GridSearchCV(xgbc, xgbc_param,cv=5, return_train_score=True,scoring = 'f1_macro' )
grid_xgb.fit(X_train,y_train)

print(f'Best Mean Cross Validation Score is {grid_xgb.best_score_}')
print(f'Train score is {grid_xgb.score(X_train,y_train)}')
print(f'Test score is {grid_xgb.score(X_test,y_test)}')

Best Mean Cross Validation Score is 0.6969503509216302
Train score is 0.9038958352496665
Test score is 0.6936071866033776


# Naive Bayes

In [12]:
#Naive Bayes
from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.naive_bayes import MultinomialNB

skf = StratifiedKFold(n_splits=10)
params = {}
nb = MultinomialNB()
gs = GridSearchCV(nb, cv=skf, param_grid=params, return_train_score=True, scoring = 'f1_macro')
gs.fit(X_train,y_train)
print(f'Best Mean Cross Validation Score is {gs.best_score_}')
print(f'Train score is {gs.score(X_train,y_train)}')
print(f'Test score is {gs.score(X_test,y_test)}')

Best Mean Cross Validation Score is 0.6460582287056138
Train score is 0.7926656333617976
Test score is 0.642445420149807


# Hash Vectorization

In [13]:
#Hash vectorizer for train data
from sklearn.feature_extraction.text import HashingVectorizer
cv=HashingVectorizer()
X=cv.fit_transform(l)
y=a.iloc[:,1].values

In [14]:
#Hash vectorization for test data
from sklearn.feature_extraction.text import HashingVectorizer
cv=HashingVectorizer()
X_t=cv.fit_transform(d)

In [15]:
# splitting trainning data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [16]:
# logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(max_iter = 100000).fit(X_train, y_train)

cv_scores = cross_val_score(clf, X_train, y_train,scoring='f1_macro')

# Mean Cross validation Score
print("Mean Cross-validation scores: {}".format(cv_scores.mean()))
print()

# Print Co-efficients
print("Logistic.coef_:", clf.coef_)
print("Logistic.intercept_:", clf.intercept_)

# Check test data set performance
print("Logistic Performance: ", clf.score(X_test,y_test))

Mean Cross-validation scores: 0.6720194826562751

Logistic.coef_: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Logistic.intercept_: [ 0.90860639 -0.10149602 -0.80711037]
Logistic Performance:  0.7800546448087432


# Submission 

In [ ]:
test_data_labels = clf.predict(X_t)

# Create predictions to be submitted!
pd.DataFrame({'id': b1.id, 'Target': test_data_labels}).to_csv('solution_base.csv', index =False)  
print("Done :D")